## 3.9 Save some text into a file corpus.txt. Define a function load(f) that reads from the file named in its sole argument, and returns a string containing the text of the file.

### (a).  Use nltk.regexp_tokenize() to create a tokenizer that tokenizes the various kinds of punctuation in this text. Use one multi-line regular expression, with inline comments, using the verbose flag (?x).

In [136]:
import nltk, re, pprint
from nltk import word_tokenize
from urllib import request

In [137]:
def load(f):
    my_text = open(f).read()
    return my_text

text = load("corpus.txt")

pattern_a = r'''(?x)           # set flag to allow verbose regexps
          [^a-zA-Z0-9\s_]    # searches for non-alphanumeric characters.
'''

all_puncts = nltk.regexp_tokenize(text, pattern_a)
all_puncts

['$', '.', '/', '/', '.']

### (b). Use nltk.regexp_tokenize() to create a tokenizer that tokenizes the following kinds of expression: monetary amounts; dates; names of people and organizations.

In [138]:
pattern = r'''(?x)    # set flag to allow verbose regexps
      \$\d+(?:\.\d+)? 
    | \d{1,2}/\d{1,2}/\d{4}
    | [A-Z][a-z]+(?=\s[A-Z])(?:\s[A-Z][a-z]+)+
'''  
nltk.regexp_tokenize(text, pattern)

['Jane Smith', '$13.04', '09/09/1994', 'Pike Place Fish Market']

## 3.43 With the help of a multilingual corpus such as the Universal Declaration of Human Rights Corpus (nltk.corpus.udhr), and NLTK's frequency distribution and rank correlation functionality (nltk.FreqDist, nltk.spearman_correlation), develop a system that guesses the language of a previously unseen text. For simplicity, work with a single character encoding and just a few languages.

In [172]:
import re
from nltk import *
from nltk.corpus import udhr
from nltk.corpus import genesis
from nltk import spearman_correlation
from nltk.corpus import gutenberg

languages = []

def target_text_prep(text):

    mystery_text = [list(word.lower()) for word in text if word.isalpha()]
    mystery_text = [item for sublist in mystery_text for item in sublist]
    fd_mystery_text = FreqDist(mystery_text)
    mystery_ranks = list(ranks_from_sequence(fd_mystery_text))

    return mystery_ranks

def language_pool_prep(fids):

    languages = [fileid for fileid in fids if re.findall('Latin1', fileid)] 
    udhr_corpus = [[list(word.lower()) for word in udhr.words(language) if word.isalpha()] for language in languages]
    udhr_corpus = [[item for sublist in language for item in sublist] for language in udhr_corpus]
    languages = list(enumerate(languages))
    language_freq_dists = [FreqDist(language) for language in udhr_corpus]
    language_ranks = [list(ranks_from_sequence(dist)) for dist in language_freq_dists]
    return languages, language_ranks

def spearman(mystery_ranks, language_ranks):

    spearman_numbers = [] 
    for language in language_ranks:
        number = spearman_correlation(language, mystery_ranks)
        spearman_numbers.append(number)

    return spearman_numbers



fids = ['French_Francais-Latin1', 'Spanish-Latin1', 'German_Deutsch-Latin1', 'English-Latin1']
text = gutenberg.words('austen-emma.txt')

languages, language_ranks = language_pool_prep(fids)
mystery_ranks = target_text_prep(text)
spearman_numbers = spearman(mystery_ranks, language_ranks)
zipped = list(zip(languages, spearman_numbers))
answer = sorted(zipped, key=lambda x: x[1])

print(answer)

[((0, 'French_Francais-Latin1'), -0.11264822134387353), ((2, 'German_Deutsch-Latin1'), 0.006153846153846176), ((1, 'Spanish-Latin1'), 0.21652173913043482), ((3, 'English-Latin1'), 0.44341880341880346)]
